In [46]:
import numpy as np
import pandas as pd
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [11]:
!pip install memory-profiler

In [12]:
! pip install mlxtend

In [37]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
import time
import memory_profiler

### Пояснение метрик:

1. **Support (Поддержка)**  
   - **Общее значение**: Показывает, как часто встречается набор симптомов в данных.  
   - **В данной задаче**: Высокая поддержка указывает на распространенное сочетание симптомов у пациентов. Например, если поддержка правила `{Кашель, Лихорадка} → {Боль в горле}` равна 0.15, это означает, что 15% пациентов имеют все три симптома.

2. **Confidence (Доверие/Уверенность)**  
   - **Общее значение**: Вероятность появления правой части правила (consequents), если наблюдается левая часть (antecedents).  
   - **В данной задаче**: Если правило `{Кашель, Лихорадка} → {Боль в горле}` имеет доверие 0.8, это значит, что в 80% случаев у пациентов с кашлем и лихорадкой также наблюдается боль в горле.

3. **Lift (Подъем)**  
   - **Общее значение**: Показывает, насколько появление последствий (consequents) связано с появлением предпосылок (antecedents) по сравнению со случайным распределением. Значение >1 указывает на положительную зависимость.  
   - **В данной задаче**: Если lift для `{Кашель, Лихорадка} → {Боль в горле}` равен 1.5, это значит, что пациенты с кашлем и лихорадкой на 50% чаще имеют боль в горле, чем можно было бы ожидать при случайном распределении симптомов.

4. **Leverage (Рычаг)**  
   - **Общее значение**: Измеряет разницу между ожидаемой и фактической встречаемостью правил. Значения >0 указывают на наличие зависимости.  
   - **В данной задаче**: Высокий рычаг для `{Одышка, Лихорадка} → {Кашель}` может указывать на специфическую связь между этими симптомами, что важно для диагностики COVID-19.

5. **Conviction (Убежденность)**  
   - **Общее значение**: Показывает, насколько часто правило выполняется по сравнению с тем, если бы оно не выполнялось. Conviction >1 указывает на сильную зависимость.  
   - **В данной задаче**: Например, если conviction `{Насморк} → {Кашель}` равно 2.0, это означает, что пациенты с насморком в 2 раза реже НЕ имеют кашель, чем можно было бы ожидать случайно.

Эти метрики помогают выявить закономерности среди симптомов, что может быть полезно для диагностики и анализа заболеваний.


In [43]:
def generate_patient_data():
    symptoms = ["Кашель", "Лихорадка", "Насморк", "Боль в горле", "Одышка", "Головная боль", "Тошнота"]

    flu = ["Кашель", "Боль в горле"]
    cold = ["Насморк", "Кашель", "Головная боль"]
    covid = ["Лихорадка", "Одышка", "Кашель", "Головная боль"]
    food_poisoning = ["Тошнота", "Головная боль", "Лихорадка"]

    flu_prob = 0.4
    cold_prob = 0.3
    covid_prob = 0.2
    food_poisoning_prob = 0.1

    num_patients = 1000
    data = []

    for _ in range(num_patients):
        patient_symptoms = {symptom: 0 for symptom in symptoms}

        num_random_symptoms = np.random.randint(0, 2)
        random_symptoms = np.random.choice(symptoms, num_random_symptoms, replace=False)

        for symptom in random_symptoms:
            patient_symptoms[symptom] = 1

        if np.random.rand() < flu_prob:
            for symptom in flu:
                patient_symptoms[symptom] = 1
        if np.random.rand() < cold_prob:
            for symptom in cold:
                patient_symptoms[symptom] = 1
        if np.random.rand() < covid_prob:
            for symptom in covid:
                patient_symptoms[symptom] = 1
        if np.random.rand() < food_poisoning_prob:
            for symptom in food_poisoning:
                patient_symptoms[symptom] = 1

        data.append(patient_symptoms)

    return pd.DataFrame(data)

df = generate_patient_data()
df.head()

,Кашель,Лихорадка,Насморк,Боль в горле,Одышка,Головная боль,Тошнота
0,1,0,1,1,0,1,0
1,0,0,0,0,0,0,0
2,1,1,0,0,1,1,0
3,1,1,0,1,1,1,0
4,1,0,0,1,0,0,0


In [44]:
import pandas as pd
import time
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
import tracemalloc

df = df.astype(bool)

# Apriori
tracemalloc.start()
start_time = time.time()

frequent_itemsets_apriori = apriori(df, min_support=0.01, use_colnames=True)

apriori_time = time.time() - start_time
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

rules_apriori = association_rules(frequent_itemsets_apriori, metric="confidence", min_threshold=0.5)
rules_apriori = rules_apriori[["antecedents", "consequents", "support", "confidence", "lift", "leverage", "conviction"]]
rules_apriori = rules_apriori[rules_apriori["antecedents"].apply(len) >= 2]

print(f"Apriori time: {apriori_time:.5f} seconds")
print(f"Number of rules (Apriori): {len(rules_apriori)}")
print(f"Peak memory used by Apriori: {peak / 10**6:.5f} MB")


# FP-Growth
tracemalloc.start()
start_time = time.time()

frequent_itemsets_fpgrowth = fpgrowth(df, min_support=0.02, use_colnames=True)

fpgrowth_time = time.time() - start_time
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# Генерация ассоциативных правил
rules_fpgrowth = association_rules(frequent_itemsets_fpgrowth, metric="confidence", min_threshold=0.5)
# Оставляем только нужные метрики
rules_fpgrowth = rules_fpgrowth[["antecedents", "consequents", "support", "confidence", "lift", "leverage", "conviction"]]
# Оставляем правила, где 2 и больше компонент
rules_fpgrowth = rules_fpgrowth[rules_fpgrowth["antecedents"].apply(len) >= 2]

print(f"\nFP-Growth time: {fpgrowth_time:.5f} seconds")
print(f"Number of rules (FP-Growth): {len(rules_fpgrowth)}")
print(f"Peak memory used by FP-Growth: {peak / 10**6:.5f} MB")

print("\nПример правил (FP-Growth):")
rules_fpgrowth.sample(10)


Apriori time: 0.04301 seconds
Number of rules (Apriori): 381
Peak memory used by Apriori: 0.53404 MB

FP-Growth time: 3.44201 seconds
Number of rules (FP-Growth): 243
Peak memory used by FP-Growth: 0.39100 MB

Пример правил (FP-Growth):


,antecedents,consequents,support,confidence,lift,leverage,conviction
275,"(Тошнота, Кашель, Одышка)","(Лихорадка, Головная боль)",0.039,1.000000,3.333333,0.027300,inf
145,"(Насморк, Одышка, Боль в горле)","(Кашель, Головная боль)",0.051,1.000000,2.087683,0.026571,inf
266,"(Тошнота, Одышка)","(Лихорадка, Кашель)",0.039,0.951220,3.458980,0.027725,14.862500
229,"(Тошнота, Насморк)","(Лихорадка, Кашель, Головная боль)",0.039,0.696429,2.618153,0.024104,2.417882
65,"(Кашель, Одышка)",(Лихорадка),0.203,0.867521,2.589616,0.124610,5.019677
89,"(Кашель, Одышка, Боль в горле)",(Лихорадка),0.095,0.805085,2.403238,0.055470,3.411739
253,"(Лихорадка, Кашель, Насморк, Тошнота)","(Головная боль, Боль в горле)",0.022,0.564103,2.442002,0.012991,1.764176
121,"(Лихорадка, Насморк, Одышка)",(Головная боль),0.092,1.000000,1.848429,0.042228,inf
74,"(Лихорадка, Одышка)","(Кашель, Головная боль)",0.203,0.990244,2.067315,0.104805,53.402500
184,"(Тошнота, Головная боль)","(Лихорадка, Кашель)",0.083,0.633588,2.303956,0.046975,1.978646


In [45]:
def create_metric_df(rules_df, metric_name, n=5):
    df = (
        rules_df.sample(n=n)
        [['antecedents', 'consequents', metric_name]]
    )
    df['Antecedents'] = df['antecedents'].apply(lambda x: ', '.join(x))
    df['Consequents'] = df['consequents'].apply(lambda x: ', '.join(x))
    df = df[['Antecedents', 'Consequents', metric_name]]
    df.columns = ['Antecedents', 'Consequents', metric_name.capitalize()]
    return df.reset_index(drop=True)

random_support_df = create_metric_df(rules_fpgrowth, 'support')
random_confidence_df = create_metric_df(rules_fpgrowth, 'confidence')
random_lift_df = create_metric_df(rules_fpgrowth, 'lift')
random_leverage_df = create_metric_df(rules_fpgrowth, 'leverage')
random_conviction_df = create_metric_df(rules_fpgrowth, 'conviction')


def print_metric_df(df, format_col, format_str):
    display(df.style.format({format_col: format_str}))

print_metric_df(random_support_df, 'Support', '{:.4f}')

print_metric_df(random_confidence_df, 'Confidence', '{:.4f}')

print_metric_df(random_lift_df, 'Lift', '{:.4f}')

print_metric_df(random_leverage_df, 'Leverage', '{:.6f}')

print_metric_df(random_conviction_df, 'Conviction', '{:.4f}')

,Antecedents,Consequents,Support
0,"Насморк, Кашель, Головная боль, Одышка",Лихорадка,0.0920
1,"Лихорадка, Кашель, Насморк, Тошнота",Боль в горле,0.0220
2,"Насморк, Тошнота",Кашель,0.0530
3,"Лихорадка, Тошнота",Головная боль,0.1170
4,"Тошнота, Кашель, Боль в горле","Лихорадка, Головная боль",0.0470


,Antecedents,Consequents,Confidence
0,"Насморк, Кашель, Боль в горле",Головная боль,0.8780
1,"Насморк, Тошнота","Кашель, Головная боль",0.9464
2,"Тошнота, Одышка",Кашель,0.9512
3,"Лихорадка, Насморк, Тошнота, Боль в горле",Кашель,1.0000
4,"Кашель, Боль в горле",Головная боль,0.5754


,Antecedents,Consequents,Lift
0,"Насморк, Головная боль, Одышка","Лихорадка, Кашель",3.1561
1,"Кашель, Одышка, Боль в горле",Головная боль,1.5821
2,"Лихорадка, Тошнота",Головная боль,1.8484
3,"Лихорадка, Кашель, Тошнота",Головная боль,1.8484
4,"Лихорадка, Кашель, Одышка",Головная боль,1.8484


,Antecedents,Consequents,Leverage
0,"Тошнота, Кашель, Насморк","Лихорадка, Головная боль",0.023100
1,"Насморк, Тошнота, Боль в горле",Лихорадка,0.013960
2,"Головная боль, Одышка, Боль в горле","Насморк, Кашель",0.015852
3,"Лихорадка, Тошнота, Одышка",Головная боль,0.018819
4,"Лихорадка, Одышка, Боль в горле",Головная боль,0.043605


,Antecedents,Consequents,Conviction
0,"Лихорадка, Насморк","Кашель, Головная боль",23.2713
1,"Тошнота, Насморк","Лихорадка, Кашель, Головная боль",2.4179
2,"Кашель, Одышка","Лихорадка, Головная боль",5.2839
3,"Кашель, Головная боль",Лихорадка,1.4955
4,"Лихорадка, Тошнота, Головная боль, Боль в горле, Насморк",Кашель,inf
